# Patch community name

In [ ]:
from embedding_search.community_map import download_datafile

df = download_datafile()
df.to_parquet("tmp/community.parquet")

id_to_community_name = {row.orcid: row.community_name for row in df.itertuples()}
id_to_email = {row.orcid: row.email for row in df.itertuples()}

In [ ]:
df.orcid.tolist()

In [ ]:
import json
from pathlib import Path
from embedding_search.data_model import Author
from tqdm import tqdm

AUTHORS_DIR = Path("authors/")
authors_path = AUTHORS_DIR.glob("*.json")


def patch_author(json_path: Path) -> None:
    """Patch author JSON to include community name and email."""

    with open(json_path, "r") as f:
        author = json.load(f)

    author = Author(**author)
    author.email = id_to_email[author.orcid]
    author.community_name = id_to_community_name[author.orcid]
    author.save(AUTHORS_DIR / f"{author.orcid}.json")


for file in tqdm(authors_path):
    patch_author(file)

# Patch cited-by in the database

In [ ]:
from pathlib import Path
from tqdm import tqdm
from embedding_search.crossref import query_crossref
from embedding_search.vector_store import get_author

authors_path = Path("./authors/").glob("*.json")
last_processed = "0000-0002-5769-7094"


short_listed_authors = []
start = False
for author_file in list(authors_path):
    if author_file.stem == last_processed:
        start = True
    if start:
        short_listed_authors.append(author_file)

In [ ]:
short_listed_authors

In [ ]:
for author_file in tqdm(short_listed_authors):
    print(author_file)
    author = get_author(author_file.stem)

    for article in author.articles:
        _, cited_by = query_crossref(article.doi)
        if cited_by:
            article.cited_by = cited_by

    author.save(author_file)